In [1]:
import numpy as np
import math
from numba import jit, prange, cuda

In [2]:
import numba
numba.__version__

'0.51.2'

### test

In [18]:
@cuda.jit
def plusKernel(a, b):
    r, c = cuda.grid(2)
    if r < 100 and c < 100:
        a[r][c] += b[r]

a = np.zeros((100, 100), dtype=float)
b = np.empty((100, 100), dtype=int)
b.fill(100)

block_size = (32,32)
grid_size = (int(100 / 32) + 1, int(100 / 32) + 1)

plusKernel[grid_size, block_size](a, b)
a

TypingError: ignored

### read data

In [20]:
from numba import cuda, jit
import numba
import numpy as np
import math

INF = 123456789
f = open("graph.txt", "r")
vertices = int(f.readline())
g_pyObj = []
# print(vertices)
for i in range(vertices):
    g_pyObj.append(list(map(int, f.readline().split())))
f.close()

# sizeOfG = np.zeros(vertices, dtype=int)
g = np.empty((vertices, vertices), dtype=int)
g.fill(INF)
for i in range(vertices):
    for j in range(len(g_pyObj[i])):
        g[i][j] = g_pyObj[i][j]
    # sizeOfG[i] = len(g_pyObj[i])

bet = np.zeros((vertices, vertices), dtype=float)

### jit

In [37]:
@cuda.jit
# def bfs_kernel(g: np.array, bet: np.array, vertices: int, level: np.array, parents: np.array):
def bfs_kernel(vertices: int):
    bas = 3
    cas = 3
    sV = cuda.shared.array(vertices, dtype=g.dtype)
    sQ = cuda.shared.array(vertices, dtype=g.dtype)
    sP = cuda.shared.array(2, dtype=g.dtype) 
    idx = cuda.threadIdx.x
    if cuda.blockIdx.x > vertices:
        return

    if idx < vertices:
        sV[idx] = 0
        sQ[idx] = 0

    if idx == cuda.blockDim.x - 1:
        sP[0] = 0
        sP[1] = 1
        sQ[0] = cuda.blockIdx.x
        sV[cuda.blockIdx.x] = 1

    while sP[0] < sP[1]:
        
        rangeHandle = sP[1] - sP[0]
        if idx < rangeHandle:
            l = sP[0]; r = sP[1]
            u = sQ[l + idx]
            for vIdx in range(vertices):
                v = g[u][vIdx]
                if not sV[v]: # meet childs
                    cuda.atomic.add(sV, v, 1)
                    cuda.atomic.add(sQ, r, v)
                    cuda.atomic.add(sP, 1, 1)
                    cuda.atomic.add(level, v, level[u] + 1)
                else: # meet parents
                    if level[u] == level[v] + 1:
                        cuda.atomic.add(parents, u, 1)    
        cuda.syncthreads()
        if idx == 0:
                sP[0] = r

### show rs

In [38]:
block_size = 32
grid_size = math.ceil(vertices / block_size)

level = np.empty(vertices, dtype=int)
level.fill(INF)
parents = np.zeros(vertices, dtype=int)

bfs_kernel[grid_size, block_size](vertices)
bfs_kernel[grid_size, block_size](g, bet, vertices, level, parents)

level

TypingError: ignored